In [1]:
from qdrant_client import QdrantClient
import os
import json
from langchain_openai import AzureOpenAIEmbeddings
from qdrant_client import QdrantClient
client = QdrantClient("http://localhost:6333", timeout=300)

In [2]:
os.environ["OPENAI_API_KEY"] = ""
os.environ["AZURE_OPENAI_ENDPOINT"] = ""
os.environ["OPENAI_API_VERSION"] = ""
os.environ["OPENAI_API_TYPE"] = ""

In [3]:
def load_queries(dataset):
    queries = {}

    with open(dataset, "r") as file:
        for line in file:
            row = json.loads(line)
            queries[row["_id"]] = row["text"]
    return queries

In [4]:
dataset = "./queries.jsonl"
queries = load_queries(dataset)

In [12]:
#values = list(queries.values())[:10]
#print(len(values))

10


In [6]:

# this function encodes a query into a dense vector using openai embeddings
def encode_dense_query(query):
    embeddings = AzureOpenAIEmbeddings(
        model= "text-embedding-ada-002",  
        azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
        openai_api_version=os.environ["OPENAI_API_VERSION"],
        openai_api_key=os.environ["OPENAI_API_KEY"],
    )
    query_vector = embeddings.embed_query(query)
    return query_vector

In [7]:
def dense_search(dense_query):
    results = client.query_points(
        collection_name="quora_collection",
        query=dense_query,
        using="openai", 
        with_payload=True,
        limit=10
    )
    return results

In [8]:
with open ("./dense.jsonl", "w") as dense_out:
    for idx,query in queries.items():
        dense_vector = encode_dense_query(query)
        dense_results = dense_search(dense_vector)
        print("Dense Results:", dense_results)

        dense_output = {"query_id": idx,"results":
                        [{"doc_id": point.id,"score": point.score}
                         for point in dense_results.points]}
        dense_out.write(json.dumps(dense_output) + "\n")


Dense Results: points=[ScoredPoint(id=2739791929, version=7758, score=0.9013387, payload={'page_content': 'Every time a pandemic occurs, dozens of theories emerge to attribute the origin of the event to different facts. The COVID-19 pandemic that has hit virtually all the globe has been no exception. What is known so far about the origin of the virus that causes COVID 19? The first investigations on the origin of this disease have determined that it is a new type of virus, the origin of which is most likely zoonotic.', 'metadata': None}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=9326394684, version=8077, score=0.89683723, payload={'page_content': 'The emerging coronavirus disease (COVID-19) swept across the world, affecting more than 200 countries and territories. Genomic analysis suggests that the COVID-19 virus originated in bats and transmitted to humans through unknown intermediate hosts in the Wuhan seafood market, China, in December of 2019. This virus belong